In [1]:
from datasets import load_dataset

dataset = load_dataset("ccdv/arxiv-summarization")
dataset

No config specified, defaulting to: arxiv-summarization/section
Found cached dataset arxiv-summarization (C:/Users/JustinDu/.cache/huggingface/datasets/ccdv___arxiv-summarization/section/1.0.0/fa2c9abf4312afb8660ef8e041d576b8e3943ea96ae771bd3cd091b5798e7cc3)


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['article', 'abstract'],
        num_rows: 203037
    })
    validation: Dataset({
        features: ['article', 'abstract'],
        num_rows: 6436
    })
    test: Dataset({
        features: ['article', 'abstract'],
        num_rows: 6440
    })
})

In [2]:
data_train = dataset['train']
data_test = dataset['test']
data_val = dataset['validation']
data_test

Dataset({
    features: ['article', 'abstract'],
    num_rows: 6440
})

In [3]:
data_test = data_test.shard(num_shards=644, index=0)
data_val = data_val.shard(num_shards=403, index=0)

In [4]:
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM
from transformers import DataCollatorForSeq2Seq

checkpoint = 'facebook/bart-large-cnn'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [5]:
max_length = 1024
max_output_length = 1024

def tokenize_function(data):

    model_inputs = tokenizer(
        data["article"],
        truncation=True,
        padding='max_length',
        max_length=max_length,
        return_tensors='pt'
    )

    labels = tokenizer(
        data['abstract'],
        truncation=True,
        padding='max_length',
        max_length=max_output_length,
        return_tensors='pt'
    )
    model_inputs["labels"] = labels["input_ids"]
    model_inputs["labels_mask"] = labels["attention_mask"]

    return model_inputs

In [6]:
tok = data_test.map(tokenize_function, batched=True)
tok_val = data_val.map(tokenize_function, batched=True)
tok

Loading cached processed dataset at C:/Users/JustinDu/.cache/huggingface/datasets/ccdv___arxiv-summarization/section/1.0.0/fa2c9abf4312afb8660ef8e041d576b8e3943ea96ae771bd3cd091b5798e7cc3\cache-d3acfee19d75ced6.arrow
Loading cached processed dataset at C:/Users/JustinDu/.cache/huggingface/datasets/ccdv___arxiv-summarization/section/1.0.0/fa2c9abf4312afb8660ef8e041d576b8e3943ea96ae771bd3cd091b5798e7cc3\cache-467d447a8fcfde69.arrow


Dataset({
    features: ['article', 'abstract', 'input_ids', 'attention_mask', 'labels', 'labels_mask'],
    num_rows: 10
})

In [7]:
tok = tok.remove_columns(
    data_test.column_names
)

tok_val = tok_val.remove_columns(
    data_val.column_names
)

In [8]:
import evaluate
import nltk


nltk.download("punkt")
rouge_score = evaluate.load('rouge')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\JustinDu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [9]:
from nltk.tokenize import sent_tokenize

def three_sentence_summary(text):
    return "\n".join(sent_tokenize(text)[:3])

In [10]:
print(three_sentence_summary(data_train[2]["article"]))

the transport properties of nonlinear non - equilibrium dynamical systems are far from well - understood@xcite .
consider in particular so - called ratchet systems which are asymmetric periodic potentials where an ensemble of particles experience directed transport@xcite .
the origins of the interest in this lie in considerations about extracting useful work from unbiased noisy fluctuations as seems to happen in biological systems@xcite .


In [11]:
def evaluate_baseline(dataset, metric):
    summaries = [three_sentence_summary(text) for text in dataset["article"]]
    return metric.compute(predictions=summaries, references=dataset["abstract"])

In [12]:
import pandas as pd

score = evaluate_baseline(data_val, rouge_score)


In [13]:
score

{'rouge1': 0.24811990022937655,
 'rouge2': 0.056522925237472044,
 'rougeL': 0.15155617919514053,
 'rougeLsum': 0.2233212938696525}

In [14]:
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
rouge_dict = dict((rn, round(score[rn]* 100, 2)) for rn in rouge_names)
rouge_dict

{'rouge1': 24.81, 'rouge2': 5.65, 'rougeL': 15.16, 'rougeLsum': 22.33}

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [15]:
extra_args = {
    "gradient_accumulation_steps": 4,
    "gradient_checkpointing": True,
    "fp16": "True",
    "optim": "adafactor"

}

In [16]:
from transformers import Seq2SeqTrainingArguments

batch_size = 1
num_train_epochs = 8
logging_steps = 10
training_args = Seq2SeqTrainingArguments(
    output_dir='arxiv-bart',
    evaluation_strategy="epoch",
    learning_rate=5.6e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=10,
    num_train_epochs=num_train_epochs,
    predict_with_generate=True,
    logging_steps=logging_steps,
    label_names=['labels'],
    # push_to_hub=True,
    resume_from_checkpoint=True,
    save_steps=1,
    # no_cuda=True,
    **extra_args
)

In [17]:
import numpy as np
from nltk.tokenize import sent_tokenize


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # Decode generated summaries into text
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    # Decode reference summaries into text
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # ROUGE expects a newline after each sentence
    decoded_preds = ["\n".join(sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(sent_tokenize(label.strip())) for label in decoded_labels]
    # Compute ROUGE scores
    result = rouge_score.compute(
        predictions=decoded_preds, references=decoded_labels, use_stemmer=True
    )
    # Extract the median scores
    result = {key: value* 100 for key, value in result.items()}
    return {k: round(v, 4) for k, v in result.items()}

In [18]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    training_args,
    train_dataset=tok,
    eval_dataset=tok_val,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

Using cuda_amp half precision backend


In [19]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: labels_mask. If labels_mask are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 10
  Num Epochs = 8
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 4
  Total optimization steps = 16
  Number of trainable parameters = 406290432


  0%|          | 0/16 [00:00<?, ?it/s]

You're using a BartTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Saving model checkpoint to arxiv-accelerate\checkpoint-1
Configuration saved in arxiv-accelerate\checkpoint-1\config.json
Model weights saved in arxiv-accelerate\checkpoint-1\pytorch_model.bin
tokenizer config file saved in arxiv-accelerate\checkpoint-1\tokenizer_config.json
Special tokens file saved in arxiv-accelerate\checkpoint-1\special_tokens_map.json


KeyboardInterrupt: 

In [ ]:
trainer.evaluate()

In [ ]:
trainer.push_to_hub(commit_message="step 110 eval", tags="summarization")

In [ ]:
# samples = tok['test'][:2000]
# samples = {k: v for k, v in samples.items() if k not in ['abstract', 'article']}
# batch = data_collator(samples)

In [ ]:
# chunk = 500
# tok_abs = []

# for i in (data_test[pos:pos + chunk] for pos in range(0, len(data_test), chunk)):
#     tok_abs.append(tokenizer(i['abstract'], truncation=True, padding=True, return_tensors='tf'))


In [ ]:
# tok_art = []
# for i in (data_test[pos:pos + chunk] for pos in range(0, len(data_test), chunk)):
#     tok_art.append(tokenizer(i['article'], truncation=True, ))

In [ ]:
# import torch
# tok = torch.cat(tok_abs, dim=1)


In [ ]:
# test_tok_abs = tokenizer(data_train['article'], truncation=True, padding='max_length')